In [ ]:
# remove this
import sys
sys.path.append("/Users/harrison37/Work/alpine/ascent/install-debug/python-modules/")

In [ ]:
# cleanup any old results
!./cleanup.sh

In [ ]:
# Ascent imports
# ascent + conduit imports
import conduit
import conduit.blueprint
import ascent

import numpy as np

# Jupyter imports
from IPython.display import Image
# helper we use when displaying results in the notebook
img_display_width = 500

In [ ]:
# Setup Actions to render our input datasets

# setup actions
actions = conduit.Node()
add_act = actions.append()
add_act["action"] = "add_scenes"
scenes = add_act["scenes"]
scenes["s1/plots/p1/type"] = "pseudocolor";
scenes["s1/plots/p1/field"] = "iters";
scenes["s1/image_prefix"] = "out_julia_";
scenes["s1/image_width"] = 500
scenes["s1/image_height"] = 500

# print our full actions tree
print(actions.to_yaml())

# Sweep the juila set to create a sample time varying
# data set

nsteps = 20

# we will generate and rendering juila set examples linearly 
# interpolated between to known boxes

# starting box
extents_start = np.array( [-2.0, 2.0,  # x_min, x_max
                          -2.0, 2.0], # y_min, y_max
                           dtype=np.float64)

# ending box
extents_end = np.array( [-0.8, -0.4,  # x_min, x_max
                         0.0, 0.4],  # y_min, y_max
                         dtype=np.float64)

extents_delta = extents_end - extents_start

a = ascent.Ascent()
a.open()

info = conduit.Node()

for i in range(nsteps+1):
    extents = extents_start + extents_delta * (float(i) / float(nsteps))
    print("Generating example with extents {}".format(extents))
    # create new dataset
    mesh = conduit.Node()
    conduit.blueprint.mesh.examples.julia(250,   250, # nx, ny
                                         extents[0], extents[1], # x range
                                         extents[2], extents[3], # y range
                                         0.285, 0.01, # c value
                                         mesh)
    # publish mesh to ascent
    a.publish(mesh)
    # render with ascent
    a.execute(actions)
    a.info(info)
    print("Created {}".format(info["images"][0]["image_name"]))
# close ascent
a.close()

In [ ]:
Image("out_julia_0.png",width=img_display_width)

In [ ]:
pattern = "out_julia_%d.png"
data = []
for i in range(20):
    with open(pattern % i, "rb") as f:
        data.append(f.read())

In [ ]:
import ipywidgets as widgets
 
image = widgets.Image(value=data[0],
    width=500,
    height=500,
    format="jpg")
 
play = widgets.Play(value=0,
    min=0,
    max=len(data)-1,
    step=1,
    interval=500)
 
def update(change):
    index = change.owner.value
    image.value = data[index]
 
play.observe(update)
 
widgets.VBox([image, play])

In [ ]:
from ascent_tutorial_jupyter_utils import ImageSeqViewer

In [ ]:
ifiles = [ "out_julia_%d.png" % i for i in range(20) ]

ifiles


In [ ]:
print(ifiles)

In [ ]:
v = ImageSeqViewer(ifiles)

In [ ]:
dir(v)

In [ ]:
v.run()